In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from tensorflow import keras

In [4]:
train_dir = "../train"
test_dir = "../test"

print(train_dir)
print(test_dir)


../train
../test


In [5]:
# Carregue os datasets
img_size = (224, 224)  # Tamanho desejado (ex: 224x224 para Transfer Learning)
batch_size = 32        # Pode ajustar conforme sua GPU

train_ds_original = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,       # Embaralha os dados
    seed=42             # Para reprodutibilidade
)

test_ds_original = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False       # Não precisa embaralhar o teste
)

Found 6802 files belonging to 5 classes.
Found 2562 files belonging to 5 classes.


In [ ]:
# Número de classes (letras)
class_names = train_ds_original.class_names
print("Classes (letras) encontradas:", class_names)

# Função para converter RGB → Grayscale
def convert_to_grayscale(image, label):
    image = tf.image.rgb_to_grayscale(image)
    image = image / 255.0
    return image, label

# Aplica as transformações (grayscale + normalização)
train_ds = train_ds_original.map(convert_to_grayscale)
test_ds = test_ds_original.map(convert_to_grayscale)



Classes (letras) encontradas: ['A', 'E', 'I', 'O', 'U']


AttributeError: '_MapDataset' object has no attribute 'shape'

In [7]:
# Verifica o formato (agora será (224, 224, 1))
for images, labels in train_ds.take(1):
    print("\nFormato das imagens (pós-grayscale):", images.shape)  # (batch_size, 224, 224, 1)
    print("Formato de UMA imagem:", images[0].shape)  # (224, 224, 1)
    print("Rótulos do batch:", labels.numpy())  # Ex: [0, 2, 1, ...]


Formato das imagens (pós-grayscale): (32, 224, 224, 1)
Formato de UMA imagem: (224, 224, 1)
Rótulos do batch: [0 4 3 2 2 4 4 1 1 1 1 0 2 0 2 2 4 2 0 0 0 2 4 3 2 3 1 3 4 1 4 2]


In [ ]:
# Define o modelo com input_shape correto (1 canal)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(input_shape=(224,224)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=5)

d:\Nova pasta\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 122s 541ms/step - accuracy: 0.6069 - loss: 2.5008 - val_accuracy: 0.2787 - val_loss: 2.5580
Epoch 2/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 76s 354ms/step - accuracy: 0.9965 - loss: 0.0671 - val_accuracy: 0.3247 - val_loss: 2.4506
Epoch 3/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 75s 349ms/step - accuracy: 0.9990 - loss: 0.0226 - val_accuracy: 0.3064 - val_loss: 3.1294
Epoch 4/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 73s 340ms/step - accuracy: 0.9997 - loss: 0.0106 - val_accuracy: 0.3169 - val_loss: 3.2143
Epoch 5/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 79s 327ms/step - accuracy: 1.0000 - loss: 0.0063 - val_accuracy: 0.3021 - val_loss: 3.8505


In [9]:
# Avaliação no conjunto de teste
test_loss, test_accuracy = model.evaluate(test_ds)

print(f"Acurácia no teste: {test_accuracy:.2%}")
print(f"Perda no teste: {test_loss:.4f}")

81/81 ━━━━━━━━━━━━━━━━━━━━ 6s 73ms/step - accuracy: 0.1590 - loss: 4.0735
Acurácia no teste: 30.21%
Perda no teste: 3.8505


In [10]:
#etapa 10 -  TREINAMENTO DE IMAGEM EXTERNA

from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np

#caminho das imagens
caminho = '12.png'

#abrir imagem com PIL
img = Image.open(caminho)# L = grayscale

#redimensiona opara 28x28 pixls (formato do dataset Fashion MNIST)
img = img.resize((64,64))

#converter para array numpy e normalizar
img_array = np.array(img) / 255.0  #nomalizar par 0-1(preto ou branco)

#oprcional -  inverte cores se o fundo para escuro se a peça for clara
# img_array = 1 - img_array

#adiciona a dimensao de lote (batch_size=1)
img_array = img_array.reshape(1,64,64,3) #em 1 dimensao no formato 28x28

import matplotlib.pyplot as plt
plt.imshow(img_array[0], cmap='gray')
plt.title('Imagem tratada')
plt.axis('off')
plt.show()

#se o modelo for treinado com o fundo escuro precisa converter as imagens do mesmo jeito

FileNotFoundError: [Errno 2] No such file or directory: '12.png'

In [28]:
#previsao da classe
predicao = model.predict(img_array)

#idetificando o indice da classe cm maior probabilidade
indice_classe = np.argmax(predicao)

print(f'Classe prevista: {class_names[indice_classe]}')
print(f'Confirança: {predicao[0][indice_classe]*100:.2f}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Classe prevista: F
Confirança: 100.00%


In [29]:
model.save("modelo_libras2.h5")